<a href="https://colab.research.google.com/github/KashishOswal9/Automatic-License-plate-detection/blob/main/Automatic_Number_Plate_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install easyocr imutils opencv-python matplotlib


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import easyocr
import imutils
from google.colab.patches import cv2_imshow  # Use this instead of cv2.imshow
from google.colab import files

In [36]:
# Step 1: Read the image
img = cv2.imread('image1.jpg')
if img is None:
    raise FileNotFoundError("Image not found. Please check the path.")

# Step 2: Convert image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Step 3: Apply bilateral filter to reduce noise while keeping edges sharp
bfilter = cv2.bilateralFilter(gray, 11, 17, 17)

# Step 4: Perform Canny edge detection
edged = cv2.Canny(bfilter, 30, 200)

In [ ]:
# Visualize edges
plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
plt.title('Grayscale Image')
plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1,2,2)
plt.title('Edge Image')
plt.imshow(edged, cmap='gray')
plt.axis('off')
plt.show()

In [38]:
# Step 5: Find contours based on edges
keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)

# Step 6: Sort contours based on area, keep top 10 for plate detection
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

location = None

# Step 7: Loop over contours to find a quadrilateral (possible plate)
for contour in contours:
    peri = cv2.arcLength(contour, True)  # Perimeter of contour
    approx = cv2.approxPolyDP(contour, 0.018 * peri, True)  # Approximate polygon
    if len(approx) == 4:
        location = approx
        break

In [ ]:
# Step 8: If plate contour found, create mask and extract plate region
if location is None:
    print("License plate contour not detected.")
else:
    mask = np.zeros(gray.shape, np.uint8)
    cv2.drawContours(mask, [location], 0, 255, -1)

    # Bitwise AND mask and original image to get plate area
    new_image = cv2.bitwise_and(img, img, mask=mask)

    # Crop the bounding rectangle from the grayscale image for OCR
    (x, y) = np.where(mask == 255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]

    # Show detected plate region
    plt.figure(figsize=(6,4))
    plt.title("Detected License Plate Region")
    plt.imshow(cropped_image, cmap='gray')
    plt.axis('off')
    plt.show()


In [40]:
    # Step 9: Use EasyOCR to read text from the cropped plate
    reader = easyocr.Reader(['en'])
    result = reader.readtext(cropped_image)

    if len(result) == 0:
        print("No text detected on plate.")
    else:
        # Extract text with highest confidence (usually first result)
        text = result[0][-2]
        print("Detected License Plate Text:", text)

   # Step 10: Annotate the original image with detected text and rectangle
        font = cv2.FONT_HERSHEY_SIMPLEX
        res = cv2.putText(img, text=text, org=(location[0][0][0], location[1][0][1] + 60),
                          fontFace=font, fontScale=1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
        res = cv2.rectangle(res, tuple(location[0][0]), tuple(location[2][0]), (0, 255, 0), 3)

Detected License Plate Text: HR.26 BR.9044


In [ ]:
        # Show final annotated image
        plt.figure(figsize=(10,6))
        plt.title("License Plate Detection Result")
        plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()


In [ ]:
reader = easyocr.Reader(['en'])

cap = cv2.VideoCapture('car1.mp4')  # Or 0 for webcam if Colab supports

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(bfilter, 30, 200)

    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    location = None

    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.018 * peri, True)
        if len(approx) == 4:
            location = approx
            break

    if location is not None:
        mask = np.zeros(gray.shape, np.uint8)
        cv2.drawContours(mask, [location], 0, 255, -1)
        (x, y) = np.where(mask == 255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_plate = gray[x1:x2+1, y1:y2+1]

        result = reader.readtext(cropped_plate)
        if len(result) > 0:
            text = result[0][-2]
            cv2.putText(frame, text, (location[0][0][0], location[1][0][1] + 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.drawContours(frame, [location], -1, (0, 255, 0), 3)

    # Replace cv2.imshow with cv2_imshow for Colab
    cv2_imshow(frame)

    # Use waitKey delay; here, just a short delay
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [16]:

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object to save output
output_video_path = 'output_license_plate.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # For .mp4 output
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(bfilter, 30, 200)

    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    location = None

    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.018 * peri, True)
        if len(approx) == 4:
            location = approx
            break

    if location is not None:
        mask = np.zeros(gray.shape, np.uint8)
        cv2.drawContours(mask, [location], 0, 255, -1)
        (x, y) = np.where(mask == 255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_plate = gray[x1:x2+1, y1:y2+1]

        result = reader.readtext(cropped_plate)
        if len(result) > 0:
            text = result[0][-2]
            cv2.putText(frame, text, (location[0][0][0], location[1][0][1] + 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.drawContours(frame, [location], -1, (0, 255, 0), 3)

    # Write the annotated frame to output video
    out.write(frame)

    # Optional: print progress every 30 frames
    if frame_count % 30 == 0:
        print(f"Processed {frame_count} frames")

cap.release()
out.release()

print(f"Video saved as {output_video_path}")

# Download the video file to your local machine
files.download(output_video_path)


Video saved as output_license_plate.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>